In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup


# Cazoo

## Testing

In [2]:
url = 'https://www.cazoo.co.uk/cars/'
res = requests.get(url)
res

<Response [200]>

In [3]:
soup = BeautifulSoup(res.content, 'html.parser')
table = soup.find(attrs={'class':"resultsstyles__ResultsList-sc-11kjmxu-0 ezjHRs"})
tbody = table.find_all('li')
tr = tbody[3]
make_model = tr.find(attrs={'class':'vehicle-cardstyles__CustomTitle-sc-1bxv5iu-8 vEuQf'}).text
version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text
dets= tr.find_all(attrs={'data-test-id':'tags'})
version = version.split()
engine = ''.join(version[0])
trim = " ".join(version[1:])
price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text

In [4]:
split = make_model.split()
split[0]

'Skoda'

In [5]:
url = 'https://www.cazoo.co.uk/cars/?numDoors=4%2C5'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
pageselection = soup.find(attrs={'aria-label':'Pagination'})
numpages = pageselection.find_all(attrs={'class':'cui__sc-j9mjm2-0 cui__sc-j9mjm2-3 jkCaUj dUXlEO'})
int(numpages[-1].text.split()[-1])
numpages

[<li class="cui__sc-j9mjm2-0 cui__sc-j9mjm2-3 jkCaUj dUXlEO"><a data-test-id="pagination-link-2" href="/cars/?numDoors=4%2C5&amp;page=2"><span class="visuallyhidden">Go to page<!-- --> </span>2</a></li>,
 <li class="cui__sc-j9mjm2-0 cui__sc-j9mjm2-3 jkCaUj dUXlEO"><a data-test-id="pagination-link-3" href="/cars/?numDoors=4%2C5&amp;page=3"><span class="visuallyhidden">Go to page<!-- --> </span>3</a></li>,
 <li class="cui__sc-j9mjm2-0 cui__sc-j9mjm2-3 jkCaUj dUXlEO"><a data-test-id="pagination-link-90" href="/cars/?numDoors=4%2C5&amp;page=90"><span class="visuallyhidden">Go to page<!-- --> </span>90</a></li>]

## Automating

In [20]:
vehicles = []
links = {'https://www.cazoo.co.uk/cars/?numDoors=2%2C3',
         'https://www.cazoo.co.uk/cars/?numDoors=4%2C5',
        'https://www.cazoo.co.uk/cars/land-rover/?numDoors=4%2C5',
         'https://www.cazoo.co.uk/cars/land-rover/?numDoors=2%2C3',
         'https://www.cazoo.co.uk/cars/mercedes-benz/?numDoors=2%2C3',
         'https://www.cazoo.co.uk/cars/mercedes-benz/?numDoors=4%2C5',
         'https://www.cazoo.co.uk/cars/bmw/?numDoors=4%2C5',
         'https://www.cazoo.co.uk/cars/bmw/?numDoors=2%2C3',
         'https://www.cazoo.co.uk/cars/audi/?numDoors=2%2C3',
         'https://www.cazoo.co.uk/cars/audi/?numDoors=4%2C5',
         'https://www.cazoo.co.uk/cars/volkswagen/?numDoors=4%2C5',
         'https://www.cazoo.co.uk/cars/volkswagen/?numDoors=2%2C3',
         'https://www.cazoo.co.uk/cars/jaguar/?numDoors=4%2C5'
        }
for url in links:
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    soup = BeautifulSoup(res.content, 'html.parser')
    table = soup.find(attrs={'data-test-id':"results-list"})
    tbody = table.find_all('li')
    tbody = tbody[3:]
    pageselection = soup.find(attrs={'aria-label':'Pagination'})
    if pageselection != None:
        numpages = pageselection.find_all(attrs={'class':'cui__sc-j9mjm2-0 cui__sc-j9mjm2-3 jkCaUj dUXlEO'})
        n = int(numpages[-1].text.split()[-1]) + 1 
    for tr in tbody:
        make_model = tr.find(attrs={'data-test-id':'card-title'}).text
        makemod = make_model.split()
        make = makemod[0]
        model = ' '.join(makemod[1:])
        version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text
        version = version.split()
        engine = ''.join(version[0])
        trim = " ".join(version[1:])
        dets = tr.find_all(attrs={'data-test-id':'tags'})
        details = dets[0]
        details_str = str(details)  
        matches = re.findall(r'(\d[\d,]* miles|\d{4} reg|Auto|Manual|Petrol|Diesel|Hybrid|\
        Electric)', details_str)
        trans = matches[2] if 'Auto' in matches[2] or 'Manual' in matches[2] else None
        mileage = matches[0] if 'miles' in matches[0] else None
        registration = matches[1] if 'reg' in matches[1] else None
        fuel_type = next((fuel_type_keyword for fuel_type_keyword in ['Petrol', 'Diesel',
                                'Hybrid', 'Electric'] if fuel_type_keyword in matches), None)
        price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text
        num_door = 5 if "numDoors=4" in url else 2
        
        vehicle = {'make':make, 'model': model, 'engine_size':engine,'trim':trim,
                   'num_doors':num_door,'mileage':mileage,'reg_year':registration,
                   'transmission_type':trans,'fuel_type':fuel_type,'price':price
              }
        vehicles.append(vehicle)
    if pageselection != None:    
        for i in range(2, n):
            url = url + '&page=' + str(i)
            res = requests.get(url)
            soup = BeautifulSoup(res.content, 'html.parser')
            table = soup.find(attrs={'data-test-id':"results-list"})

            if table != None:
                tbody = table.find_all('li')
                tbody = tbody[3:]
                for tr in tbody:
                    make_model = tr.find(attrs={'data-test-id':'card-title'}).text
                    makemod = make_model.split()
                    make = makemod[0]
                    model = ' '.join(makemod[1:])
                    version = tr.find(attrs={'class':'vehicle-cardstyles__DisplayVariant-sc-1bxv5iu-9 fxjZPj'}).text.split()
                    engine = ''.join(version[0])
                    trim = " ".join(version[1:])
                    dets = tr.find_all(attrs={'data-test-id':'tags'})
                    details = dets[0]
                    details_str = str(details)  # Convert the first object in the list to a string
                    matches = re.findall(r'(\d[\d,]* miles|\d{4} reg|Auto|Manual|Petrol|Diesel|\
                    Hybrid|Electric)', details_str)
                    trans = matches[2] if 'Auto' in matches[2] or 'Manual' in matches[2] else None
                    mileage = matches[0] if 'miles' in matches[0] else None
                    registration = matches[1] if 'reg' in matches[1] else None
                    fuel_type = next((fuel_type_keyword for fuel_type_keyword in ['Petrol', 'Diesel',
                                    'Hybrid', 'Electric'] if fuel_type_keyword in matches), None)
                    price = tr.find(attrs={'data-test-id':'card-pricing-full-price-gb'}).text
                    num_door = 5 if "numDoors=4" in url else 2

                    vehicle = {'make':make, 'model': model, 'engine_size':engine,'trim':trim,
                       'num_doors':num_door,'mileage':mileage,'reg_year':registration,
                       'transmission_type':trans,'fuel_type':fuel_type,'price':price
                      }
                    vehicles.append(vehicle)

In [21]:
cazoo = pd.DataFrame(vehicles)
cazoo

,make,model,engine_size,trim,num_doors,mileage,reg_year,transmission_type,fuel_type,price
0,BMW,1 Series,1.5,116d Sport,2,"65,221 miles",2016 reg,Manual,Diesel,"£9,400"
1,BMW,1 Series,2.0,120d M Sport,2,"77,873 miles",2015 reg,Manual,Diesel,"£9,950"
2,BMW,1 Series,1.5,118i Sport,2,"57,371 miles",2016 reg,Manual,Petrol,"£10,050"
3,BMW,1 Series,2.0,118d M Sport,2,"71,342 miles",2016 reg,Manual,Diesel,"£10,450"
4,BMW,1 Series,1.5,118i M Sport,2,"77,767 miles",2017 reg,Manual,Petrol,"was £11,000£10,700"
...,...,...,...,...,...,...,...,...,...,...
6109,Mercedes-Benz,C Class,2.0,C300 AMG Line (Premium) G-Tronic+,2,"26,700 miles",2020 reg,Auto,Petrol,"£24,550"
6110,Mercedes-Benz,E Class,3.5,E400 V6 AMG Line Edition G-Tronic+,2,"26,434 miles",2016 reg,Auto,Petrol,"£18,550"
6111,Mercedes-Benz,C Class,2.0,C300 AMG Line (Premium Plus) 7G-Tronic+,2,"51,800 miles",2016 reg,Auto,Petrol,"£18,900"
6112,Mercedes-Benz,E Class,2.0,E350 AMG Line G-Tronic+,2,"31,704 miles",2019 reg,Auto,Petrol,"£27,300"


In [22]:
cazoo.to_csv('../Data/cazoo_vehicles.csv', index = False)

# BuyaCar

## Testing

In [9]:
url2 = 'https://www.buyacar.co.uk/cars'
res2 = requests.get(url2)
res2

<Response [200]>

In [10]:
soup = BeautifulSoup(res2.content)
page=soup.find(attrs={'id':'page'})
text = page.find('script').text
mileage = re.findall(r'\bmileage\b\D*(\d+)', text)
price = re.findall(r'\bprice\b\D*([\d,]+)', text)
age = re.findall(r'\bcarDateOfReg\b\D*(\d+)', text)
cars = re.findall(r'"vehicleFullName_S":\s*"([^"]*)"', text)
gearbox = re.findall(r'"gearbox":\s*"([^"]*)"', text)
fuel = re.findall(r'"dennis_fuel_type":\s*"([^"]*)"', text)
# cars[0][0:10]
test3 = []
for i in range(0, len(cars)):
    vehicle = cars[i].split()
    make = vehicle[0]
    model = vehicle[1]
    size = vehicle[2]
    trim = ' '.join(vehicle[3:])
    test1 = {
        'Make': make,
        'Model': model,
        'engine_size': size,
        'trim': trim
    }
    test3.append(test1)
test = {
    'Mileage': mileage[1:21],
    'Age': age,
    'Gearbox': gearbox,
    'Fuel Type': fuel,
    'Price': price[1:21]
}
len(mileage), len(price), len(age), len(cars), len(gearbox), len(fuel)
#test
#test3.append(test)
test3 = pd.DataFrame(test3)
test = pd.DataFrame(test)
page1 = pd.concat([test3, test], axis=1)

## Automating

In [11]:
all_cars = []
all_info = []
miles = []
prices = []
ages = []
gears = []
fuel_types=[]
for i in range(1, 1206):
    url = 'https://www.buyacar.co.uk/cars?f[0]=vehicle_type:car&offset=' + str(i) + '&limit=20'
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')  
    page = soup.find(attrs={'id': 'page'})
    text = page.find('script').text
    
    mileage = re.findall(r'\bmileage\b\D*(\d+)', text)
    price = re.findall(r'\bprice\b\D*([\d,]+)', text)
    age = re.findall(r'\bcarDateOfReg\b\D*(\d+)', text)
    cars = re.findall(r'"vehicleFullName_S":\s*"([^"]*)"', text)
    gearbox = re.findall(r'"gearbox":\s*"([^"]*)"', text)
    fuel = re.findall(r'"dennis_fuel_type":\s*"([^"]*)"', text)
    
    cars_split = []
    for j in range(0, len(cars)):  
        vehicle = cars[j].split()
        make = vehicle[0]
        model = vehicle[1]
        size = vehicle[2]
        trim = ' '.join(vehicle[3:])
        split = {
            'Make': make,
            'Model': model,
            'engine_size': size,
            'trim': trim
        }
        cars_split.append(split)
        
    info = {
        'Age': age,
        'Gearbox': gearbox,
        'Fuel Type': fuel
    }
    miles.extend(mileage[1:21])
    prices.extend(price[2:22])
    all_cars.extend(cars_split)
    ages.extend(age)
    gears.extend(gearbox)
    fuel_types.extend(fuel)


In [12]:
all_vehicles = pd.concat([pd.DataFrame(all_cars),
                          pd.DataFrame(miles).rename(columns={0: 'mileage'}),
                          pd.DataFrame(ages).rename(columns={0: 'reg_year'}),
                          pd.DataFrame(gears).rename(columns={0: 'transmission_type'}),
                          pd.DataFrame(fuel_types).rename(columns={0: 'fuel_type'}),
                          pd.DataFrame(prices).rename(columns={0:'price'})], axis=1)
all_vehicles

,Make,Model,engine_size,trim,mileage,reg_year,transmission_type,fuel_type,price
0,Volkswagen,Golf,1.5,TSI EVO Match 5dr,33870,2019,Manual,Petrol,"15,495"
1,Volkswagen,Golf,1.0,TSI Life 5dr,3962,2022,Manual,Petrol,"20,495"
2,Volkswagen,Tiguan,1.5,TSi EVO 150 Match 5dr DSG,33997,2019,Semiauto,Petrol,"19,795"
3,Kia,Rio,1.25,ISG 2 5dr,76576,2015,Manual,Petrol,"6,395"
4,Volkswagen,Polo,1.2,TSI Match Edition 5dr,40596,2017,Manual,Petrol,"10,795"
...,...,...,...,...,...,...,...,...,...
24095,Nissan,Qashqai,1.3,DiG-T 160 Acenta Premium 5dr DCT,12185,2020,Semiauto,Petrol,"17,650"
24096,Nissan,Qashqai,1.3,DiG-T Tekna\u002B 5dr,23696,2019,Manual,Petrol,"17,650"
24097,Hyundai,IONIQ,1.6,GDi Hybrid 1st Edition 5dr DCT,8908,2019,Automatic,Hybrid,"17,695"
24098,Nissan,Qashqai,1.3,DiG-T 160 Acenta Premium 5dr DCT,8430,2020,Semiauto,Petrol,"17,700"


In [13]:
all_vehicles.to_csv('../Data/buyacar_data.csv', index=False)

# MotorPoint

## Testing

In [14]:
url3 = 'https://www.motorpoint.co.uk/used-cars'
res = requests.get(url3)
res

<Response [200]>

In [15]:
soup = BeautifulSoup(res.content, 'html.parser')
table = soup.find(attrs={'id':'plp-search-results'})
vehicles = table.find_all(attrs={'class':'vehicle-result-tile-container col-12'})
tr = vehicles[1]
title = (tr.find(attrs={'class':'title'}).text).split()
make = title[0]
model = ' '.join(title[1:])
details = tr.find(attrs={'class':'sub-title'}).text.split()
eng_size = details[0]
trim = ' '.join(details[1:])
primary = tr.find(attrs={'class':'primary-details loading-state'}).text.split()
reg_year = primary[0]
mileage = primary[4]
fuel_type = tr.find(attrs={'class':'secondary-details loading-state'}).text.split()[0]
price = tr.find(attrs={'class':'hidden-loading price-text'})


In [16]:
trim

'Turbo Elite Nav Premium 5dr'

## Automating

In [26]:
motorpoint = []
for i in range(1, 118):
    url = 'https://www.motorpoint.co.uk/used-cars?Page=' + str(i)
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    table = soup.find(attrs={'id':'plp-search-results'})
    vehicles = table.find_all(attrs={'class':'vehicle-result-tile-container col-12'})
    
    for tr in vehicles:
        title = tr.find(attrs={'class':'title'}).text.split()
        make = title[0]
        model = ' '.join(title[1:])
        trim = tr.find(attrs={'class':'sub-title'}).text
        primary = tr.find(attrs={'class':'primary-details loading-state'}).text.split()
        reg_year = primary[0]
        mileage = primary[4]
        fuel_type = tr.find(attrs={'class':'secondary-details loading-state'}).text.split()[0]
    
        price = tr.find(attrs={'class':'hidden-loading price-text'})
        if price != None:
            price = price.text
        else:
            price = tr.find(attrs={'class':'hidden-loading price-text discounted'}).text
        gears = 'Automatic' if 'Auto' in details else 'Manual' 
        
        vehicle = {'make': make, 'model':model,'engine_size':eng_size,'trim': trim,
                   'mileage':mileage,'transmission_type':gears, 'reg_year':reg_year,
                   'fuel_type':fuel_type,'price':price}
        motorpoint.append(vehicle)

In [27]:
motorpoint = pd.DataFrame(motorpoint)
motorpoint

,make,model,engine_size,trim,mileage,transmission_type,reg_year,fuel_type,price
0,Vauxhall,Corsa,1.8,1.2 Turbo Elite Nav 5dr,"37,446",Manual,2020,Petrol,"£9,999"
1,Peugeot,3008,1.8,1.2 PureTech Allure 5dr,"27,062",Manual,2020,Petrol,"£16,999"
2,Renault,Kadjar,1.8,1.3 TCE Iconic 5dr EDC,"28,272",Manual,2020,Petrol,"£13,999"
3,Seat,Ibiza,1.8,1.0 TSI 110 FR [EZ] 5dr,"7,381",Manual,2021,Petrol,"£15,499"
4,Vauxhall,Corsa,1.8,1.2 Turbo SRi Premium 5dr,"39,715",Manual,2020,Petrol,"£9,999"
...,...,...,...,...,...,...,...,...,...
4673,Audi,Q5,1.8,45 TFSI Quattro Black Edition 5dr S Tronic,"17,510",Manual,2019,Petrol,"£30,799"
4674,Seat,Ateca,1.8,1.5 TSI EVO Xperience 5dr,"7,740",Manual,2022,Petrol,"£21,699"
4675,Toyota,Aygo,1.8,1.0 VVT-i X-Trend 5dr,"7,344",Manual,2020,Petrol,"£10,099"
4676,Ford,Fiesta,1.8,1.0 EcoBoost Titanium X 5dr,"26,805",Manual,2019,Petrol,"£12,099"


In [28]:
motorpoint.to_csv('../Data/motorpoint_data.csv', index = False)